In [53]:
import re
import pandas as pd
from os import path
from collections import Counter
from glob import glob
from lxml import etree

In [54]:
#path_name = 'corpus_test/*.xml'
path_name = 'corpus_main/*.xml'

In [55]:
def canon(path_corpus):
    with open(path_corpus, encoding="utf8") as file:
        tree = etree.parse(file)
        if tree.find(".//profileDesc") is not None:
            profil = tree.find(".//profileDesc")
            if profil.get("tag") == "canon":
                return True
            else:
                return False

In [56]:
def moulinette(path_name):
    
    df_corpus = pd.DataFrame([], columns=["date", "auteur", "titre", "canon"])
    
    for doc in glob(path_name):
        
        doc_name = path.splitext(path.basename(doc))[0]
        #print(doc_name)
        date = doc_name.split("_")[0]
        auteur = doc_name.split("_")[1]
        titre = doc_name.split("_")[2]
        tag = canon(doc)
        df_roman = pd.DataFrame([(date, auteur, titre, tag)], columns=["date", "auteur", "titre", "canon"])
        df_corpus = df_corpus.append(df_roman, ignore_index = True)
        
    return df_corpus

In [79]:
df_corpus = moulinette(path_name)

In [80]:
df_corpus

,date,auteur,titre,canon
0,1811,Chateaubriand-François-Rene-de,Oeuvres-completes,True
1,1812,Kock-Paul-de,L-enfant-de-ma-femme,False
2,1815,Gay-Sophie,Anatole,False
3,1815,Gay-Sophie,Anatole,False
4,1816,Barthelemy-Hadot-Marie-Adelaide,La-Vierge-de-l-Indostan-ou-les-Portugais-au-Ma...,False
...,...,...,...,...
2956,2018,Quignard-Pascal,L-enfant-d-Ingolstadt,True
2957,2019,Dabos-Christelle,La-Passe-miroir,False
2958,2019,Murail-Marie-Aude,Sauveur-Fils,False
2959,2020,Melois-Clementine,Dehors-la-tempete,False


In [81]:
def net_auteur(liste_auteur):
    liste_result = []
    for auteur in liste_auteur:
        string_result = " "
        auteur_net = re.sub("-", " ", auteur)
        auteur_net_list = auteur_net.split(" ")
        auteur_new_order_list = auteur_net_list[-(len(auteur_net_list)-1):] + auteur_net_list[:-(len(auteur_net_list)-1)] 
        liste_result.append(string_result.join(auteur_new_order_list).lower()) 
    return liste_result

In [82]:
liste_auteur = net_auteur(list(df_corpus["auteur"]))

In [87]:
liste_auteur

['françois rene de chateaubriand',
 'paul de kock',
 'sophie gay',
 'sophie gay',
 'hadot marie adelaide barthelemy',
 'hadot marie adelaide barthelemy',
 'hadot marie adelaide barthelemy',
 'malarme charlotte de bournon',
 'malarme charlotte de bournon',
 'malarme charlotte de bournon',
 'benjamin constant',
 'hadot marie adelaide barthelemy',
 'hadot marie adelaide barthelemy',
 'hadot marie adelaide barthelemy',
 'hadot marie adelaide barthelemy',
 'hadot marie adelaide barthelemy',
 'hadot marie adelaide barthelemy',
 'charles victor arlincourt',
 'charles victor arlincourt',
 'charles victor arlincourt',
 'victor hugo',
 'claire de durfort duras',
 'd hautpoul anne marie de beaufort',
 'd hautpoul beaufort',
 'du heaume hippolyte mazier',
 'du heaume hippolyte mazier',
 'lamothe langon etienne leon de',
 'claire de duras',
 'j f t. ginouvier',
 'j f t. ginouvier',
 'j f t. ginouvier',
 'paul de kock',
 'inconnu',
 'jules janin',
 'stendhal',
 'eugene françois vidocq',
 'eugene fra

In [90]:
def net_title(liste_titre):
    liste_result = []
    for titre in liste_titre:
        temp = re.sub("-", " ", titre)
        liste_result.append(temp.lower()) 
    return liste_result

In [91]:
liste_titre = net_title(list(df_corpus["titre"]))

In [92]:
liste_titre

['oeuvres completes',
 'l enfant de ma femme',
 'anatole',
 'anatole',
 'la vierge de l indostan ou les portugais au malabar',
 'la vierge de l indostan ou les portugais au malabar',
 'la vierge de l indostan ou les portugais au malabar',
 'lancelot montagu ou le resultat des bonnes fortunes',
 'lancelot montagu ou le resultat des bonnes fortunes',
 'lancelot montagu ou le resultat des bonnes fortunes',
 'adolphe',
 'les venitiens ou le capitaine français',
 'les venitiens ou le capitaine français',
 'les venitiens ou le capitaine français',
 'laurence de sully ou l ermitage en suisse',
 'laurence de sully ou l ermitage en suisse',
 'laurence de sully ou l ermitage en suisse',
 'le solitaire',
 'le renégat',
 'le renégat',
 'han d islande',
 'ourika',
 'le page et la romance',
 'le page et la romance',
 'voyage d un jeune grec a paris',
 'voyage d un jeune grec a paris',
 'le monastere des freres noirs',
 'edouard',
 'gustave et aspais ou les victimes des prejuges de l epoque',
 'gusta

In [93]:
df_corpus["auteur"] = liste_auteur

In [94]:
df_corpus["titre"] = liste_titre

In [104]:
df_corpus

,date,auteur,titre,canon
0,1811,françois rene de chateaubriand,oeuvres completes,True
1,1812,paul de kock,l enfant de ma femme,False
2,1815,sophie gay,anatole,False
3,1815,sophie gay,anatole,False
4,1816,hadot marie adelaide barthelemy,la vierge de l indostan ou les portugais au ma...,False
...,...,...,...,...
2956,2018,pascal quignard,l enfant d ingolstadt,True
2957,2019,christelle dabos,la passe miroir,False
2958,2019,marie aude murail,sauveur fils,False
2959,2020,clementine melois,dehors la tempete,False


In [105]:
df_corpus.rename(columns = {'canon':'fabula'}, inplace = True)

In [107]:
df_corpus['fabula'].value_counts()

False    1807
True     1154
Name: fabula, dtype: int64

In [113]:
df_corpus.to_csv(r'corpus.csv', index = False, header=True)

In [108]:
auteur_prolifique = list(df_corpus["auteur"].value_counts().index.to_list())

In [114]:
nb_roman = list(df_corpus["auteur"].value_counts().to_list())

In [116]:
df_auteur_prolifique = pd.DataFrame(list(zip(auteur_prolifique, nb_roman)), columns = ['auteur', "nb_roman"])

In [119]:
df_auteur_prolifique.to_csv(r'auteurs_prolifiques.csv', index = False, header=True)

In [98]:
df_corpus["auteur"].value_counts(normalize=True)*100

delly                  3.208376
antonio san            3.039514
alexandre dumas        2.971969
honore de balzac       2.870652
georges simenon        2.701790
                         ...   
alfred jarry           0.033772
paule constant         0.033772
charles de ribelle     0.033772
francis wey            0.033772
comtesse de sannois    0.033772
Name: auteur, Length: 714, dtype: float64

In [1]:
import os

In [112]:
os.getcwd()

'C:\\Users\\jeanb\\Documents\\memoire\\dev'

In [122]:
folder_path = r"C:\Users\jeanb\Documents\memoire\dev"

In [153]:
cible_path = r"C:\Users\jeanb\Documents\memoire\dev\corpus_main_txt"

In [154]:
os.chdir(cible_path)

In [145]:
#attention fan'tome' et les nombres -> bug mais 20 occurences => a la main

In [151]:
def rename_romans(corpus_path):
    for name_folder in os.listdir(corpus_path):
        for i in range(0, len(name_folder)-4):
            if name_folder[i] == '-' and name_folder[i+1] == '(' or name_folder[i+1] in ['i','I','1','v', 'V', '2', '3', '4', '5', '6', '7', '8','9'] and name_folder[i+2] in ['i', 'I', '-','v', 'V','o'] and name_folder[i+3] in ['i', 'I', '-','v', 'V','l'] or name_folder[i+1]+name_folder[i+2]+name_folder[i+3]+name_folder[i+4] == 'tome' or name_folder[i+1]+name_folder[i+2]+name_folder[i+3]+name_folder[i+4] == 'Tome' or name_folder[i+1]+name_folder[i+2]+name_folder[i+3]+name_folder[i+4] == 'volu' or name_folder[i+1]+name_folder[i+2]+name_folder[i+3]+name_folder[i+4] == 'Volu':
                name_folder_modif = name_folder[:i]+'_'+name_folder[i+1:]
                print(name_folder, name_folder_modif)
                os.rename(name_folder, name_folder_modif)
                break;

In [155]:
rename_romans(cible_path)

1815_Gay-Sophie_Anatole-Vol-1.txt 1815_Gay-Sophie_Anatole_Vol-1.txt
1815_Gay-Sophie_Anatole-Vol-2.txt 1815_Gay-Sophie_Anatole_Vol-2.txt
1816_Barthelemy-Hadot-Marie-Adelaide_La-Vierge-de-l-Indostan-ou-les-Portugais-au-Malabar-(Tome-1).txt 1816_Barthelemy-Hadot-Marie-Adelaide_La-Vierge-de-l-Indostan-ou-les-Portugais-au-Malabar_(Tome-1).txt
1816_Barthelemy-Hadot-Marie-Adelaide_La-Vierge-de-l-Indostan-ou-les-Portugais-au-Malabar-(Tome-2).txt 1816_Barthelemy-Hadot-Marie-Adelaide_La-Vierge-de-l-Indostan-ou-les-Portugais-au-Malabar_(Tome-2).txt
1816_Barthelemy-Hadot-Marie-Adelaide_La-Vierge-de-l-Indostan-ou-les-Portugais-au-Malabar-(Tome-4).txt 1816_Barthelemy-Hadot-Marie-Adelaide_La-Vierge-de-l-Indostan-ou-les-Portugais-au-Malabar_(Tome-4).txt
1816_Bournon-Malarme-Charlotte-de_Lancelot-Montagu-ou-le-Resultat-des-bonnes-fortunes-Tome-1.txt 1816_Bournon-Malarme-Charlotte-de_Lancelot-Montagu-ou-le-Resultat-des-bonnes-fortunes_Tome-1.txt
1816_Bournon-Malarme-Charlotte-de_Lancelot-Montagu-ou-le-R

1844_Tarbe-Des-Sablons-Michelle-Catherine-Josephine-Guespereau-Mme-Sebastien-Andre_Onesie-ou-les-Soirees-de-l-abbaye-suivie-de-Enguerrand-ou-le-Duel.txt 1844_Tarbe-Des-Sablons-Michelle-Catherine-Josephine-Guespereau-Mme-Sebastien-Andre_Onesie-ou-les-Soirees-de-l-abbaye-s_ivie-de-Enguerrand-ou-le-Duel.txt
1845_Dumas-Alexandre-Pere_Le-batard-de-Mauleon-III.txt 1845_Dumas-Alexandre-Pere_Le-batard-de-Mauleon_III.txt
1845_Feval-Paul-(pere)_Les-Contes-de-nos-peres.txt 1845_Feval-Paul_(pere)_Les-Contes-de-nos-peres.txt
1845_Sue-Eugene_La-coucaratcha-Tome-I.txt 1845_Sue-Eugene_La-coucaratcha_Tome-I.txt
1845_Sue-Eugene_La-coucaratcha-Tome-II.txt 1845_Sue-Eugene_La-coucaratcha_Tome-II.txt
1845_Sue-Eugene_La-coucaratcha-Tome-III.txt 1845_Sue-Eugene_La-coucaratcha_Tome-III.txt
1845_Sue-Eugene_Le-juif-errant-tome-I.txt 1845_Sue-Eugene_Le-juif-errant_tome-I.txt
1845_Sue-Eugene_Le-juif-errant-tome-II.txt 1845_Sue-Eugene_Le-juif-errant_tome-II.txt
1845_Sue-Eugene_Paula-Monti-ou-L-Hotel-Lambert-Tome-I.

1870_Houssaye-Arsene_Les-Courtisanes-du-monde-Tome-2.txt 1870_Houssaye-Arsene_Les-Courtisanes-du-monde_Tome-2.txt
1870_Houssaye-Arsene_Les-Courtisanes-du-monde-Tome-3.txt 1870_Houssaye-Arsene_Les-Courtisanes-du-monde_Tome-3.txt
1870_Marmette-Joseph_Francois-De-Bienville--Scenes-de-la-Vie-Canadienne-au-XVII-siecle.txt 1870_Marmette-Joseph_Francois-De-Bie_ville--Scenes-de-la-Vie-Canadienne-au-XVII-siecle.txt
1870_Ponson-du-Terrail-Pierre_Rocambole-La-Corde-du-pendu-Tome-I--La-Corde-du-pendu.txt 1870_Ponson-du-Terrail-Pierre_Rocambole-La-Corde-du-pendu_Tome-I--La-Corde-du-pendu.txt
1870_Ponson-du-Terrail-Pierre_Rocambole-La-Corde-du-pendu-Tome-II--L-Homme-gris.txt 1870_Ponson-du-Terrail-Pierre_Rocambole-La-Corde-du-pendu_Tome-II--L-Homme-gris.txt
1871_Gaboriau-Emile_La-Degringolade-Tome-1.txt 1871_Gaboriau-Emile_La-Degringolade_Tome-1.txt
1871_Verne-Jules_Une-Ville-flottante.txt 1871_Verne-Jules_Une_Ville-flottante.txt
1872_Dumas-Alexandre_Robin-Hood-le-prince-des-voleurs-I.txt 1872_Dumas

1886_Villiers-de-l'Isle-Adam_L'Eve-future.txt 1886_Villiers-de-l'Isle-Adam_L'Eve-future.txt
1886_Villiers-de-l-Isle-Adam_L-Eve-future.txt 1886_Villiers-de-l-Isle-Adam_L-Eve-future.txt
1887_Greville-Henry_Nikanor.txt 1887_Gr_ville-Henry_Nikanor.txt
1887_Guy-de-Maupassant_Mont-Oriol.txt 1887_Guy-de-Maupassant_Mont-O_iol.txt
1887_Villiers-de-l'Isle-Adam_Tribulat-Bonhomet.txt 1887_Villiers-de-l'Isle-Adam_Tribulat-Bonhomet.txt
1887_Villiers-de-l-Isle-Adam-Auguste_Tribulat-Bonhomet.txt 1887_Villiers-de-l-Isle-Adam-Auguste_Tribulat-Bonhomet.txt
1888_Boisgobey-Fortune-du_L-Oeil-de-chat-Tome1.txt 1888_Boisgobey-Fortune-du_L-Oeil-de-chat_Tome1.txt
1888_Boisgobey-Fortune-du_L-Oeil-de-chat-Tome2.txt 1888_Boisgobey-Fortune-du_L-Oeil-de-chat_Tome2.txt
1888_Greville-Henry_La-seconde-mere.txt 1888_Gr_ville-Henry_La-seconde-mere.txt
1888_Stolz-Madame-de_Violence-et-bonte.txt 1888_Stolz-Madame-de__iolence-et-bonte.txt
1889_Graffigny-H-de-Le-Faure-Georges_Aventures-extraordinaires-d-un-savant-russe-Tome-

1921_Pagnol-Marcel_La-Petite-Fille-aux-yeux-sombres-suivi-de-Les-secrets-de-Dieu.txt 1921_Pagnol-Marcel_La-Petite-Fille-aux-yeux-sombres-s_ivi-de-Les-secrets-de-Dieu.txt
1922_Bordeaux-Henry_Le-fantome-de-la-rue-Michel-Ange.txt 1922_Bordeaux-Henry_Le-fa_tome-de-la-rue-Michel-Ange.txt
1922_Galopin-Arnould_Memoires-d-un-cambrioleur-retire-des-affaires.txt 1922_Galopin-Arnould_Memoires-d-un-camb_ioleur-retire-des-affaires.txt
1922_Leroux-Gaston_Le-coeur-cambriole.txt 1922_Leroux-Gaston_Le-coeur-camb_iole.txt
1922_Margueritte-Victor_Poum-(aventures-d-un-petit-garçon).txt 1922_Margueritte-Victor_Poum_(aventures-d-un-petit-garçon).txt
1922_Martin-du-Gard-Roger_Les-Thibault-Tome-I-Le-Cahier-gris.txt 1922_Martin-du-Gard-Roger_Les-Thibault_Tome-I-Le-Cahier-gris.txt
1922_Martin-du-Gard-Roger_Les-Thibault-Tome-II-Le-penitencier.txt 1922_Martin-du-Gard-Roger_Les-Thibault_Tome-II-Le-penitencier.txt
1922_Rolland-Romain_L-ame-enchantee-Annette-et-Sylvie-Volume-1.txt 1922_Rolland-Romain_L-ame-enchantee

2007_Volodine-Antoine_Songes-de-Mevlido.txt 2007_Volodine-Antoine_Songes-de-Mevlido.txt
2008_Le-Clezio-Jean-Marie-Gustave_Ritournelle-de-la-faim.txt 2008_Le-Cle_io-Jean-Marie-Gustave_Ritournelle-de-la-faim.txt
2008_Nunez-Laurent_Les-recidivistes.txt 2008_Nunez-Laurent_Les-reci_ivistes.txt
2008_Rolin-Olivier_La-Chambre-des-cartes.txt 2008_Rolin-O_ivier_La-Chambre-des-cartes.txt
2008_Rolin-Olivier_Un-chasseur-de-lions.txt 2008_Rolin-O_ivier_Un-chasseur-de-lions.txt
2010_Volodine-Andoine-(pseudonyme--Lutz-Bassmann)_Les-aigles-puent.txt 2010_Volodine-Andoine-(pseudonyme--Lutz-Bassmann)_Les-aigles-puent.txt
2010_Volodine-Antoine_Ecrivains.txt 2010_Volodine-Antoine_Ecrivains.txt
2011_Abecassis-Eliette_Et-te-voici-permise-a-tout-homme.txt 2011_Abecassis-Eliette_Et-te_voici-permise-a-tout-homme.txt
2011_Gaude-Laurent_Les-Oliviers-du-Negus.txt 2011_Gaude-Laurent_Les-O_iviers-du-Negus.txt
2012_Adam-Olivier_Les-Lisieres.txt 2012_Adam-O_ivier_Les-Lisieres.txt
2012_Damasio-Alain_Aucun-souvenir-asse

In [111]:
os.chdir(folder_path)